In [ ]:
!pip install -q --upgrade pip setuptools wheel
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 48.3 MB/s  0:00:01


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

# Config bitsandbytes (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,   # komputasi pake fp16
    bnb_4bit_use_double_quant=True,
    quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained("medalpaca/medalpaca-7b", use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    "medalpaca/medalpaca-7b",
    quantization_config=bnb_config,
    device_map="auto",   # otomatis offload ke CPU/GPU seefisien mungkin
    trust_remote_code=True
)

pl = pipeline("text-generation", model=model, tokenizer=tokenizer)
question = "What are the symptoms of diabetes?"
context = "Diabetes is a metabolic disease that causes high blood sugar. The symptoms include increased thirst, frequent urination, and unexplained weight loss."
answer = pl(f"Context: {context}\n\nQuestion: {question}\n\nAnswer: ")
print(answer)

The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


[{'generated_text': 'Context: Diabetes is a metabolic disease that causes high blood sugar. The symptoms include increased thirst, frequent urination, and unexplained weight loss.\n\nQuestion: What are the symptoms of diabetes?\n\nAnswer: \nThe symptoms of type 1 diabetes usually develop over a period of weeks or months, often without any prior symptoms.\nThe symptoms of type 2 diabetes may be vague and develop over a long period of time. Symptoms of type 2 diabetes may include:\nFrequent urination, Bed-wetting in an adult, Increased thirst, Blurred vision, Fatigue, Weakness, Hunger, Nausea, Vomiting, Diarrhea, and Weight loss, or weight gain (in spite of increased urination).'}]


In [ ]:
question = "What is your name?"
context = ""
answer = pl(f"Context: {context}\n\nQuestion: {question}\n\nAnswer: ")
print(answer)

[{'generated_text': 'Context: \n\nQuestion: What is your name?\n\nAnswer: \n\nI am Open Assistant'}]


In [ ]:
# Hitung jumlah entri 'question' yang bernilai None
none_questions = sum(1 for example in raw_dataset if example['Question'] is None)
# Hitung jumlah entri 'answer' yang bernilai None
none_answers = sum(1 for example in raw_dataset if example['Answer'] is None)

print(f"Jumlah 'question' yang bernilai None: {none_questions}")
print(f"Jumlah 'answer' yang bernilai None: {none_answers}")

Jumlah 'question' yang bernilai None: 0
Jumlah 'answer' yang bernilai None: 0


In [ ]:
# Filter dataset untuk menghapus entri yang tidak memiliki jawaban
filtered_dataset = raw_dataset.filter(lambda example: example['Answer'] is not None)
# Cek kembali jumlah entri 'answer' yang bernilai None setelah filter
none_answers_after_filter = sum(1 for example in filtered_dataset if example['Answer'] is None)

print(f"Jumlah data setelah filter: {len(filtered_dataset)}")
print(f"Jumlah 'answer' yang bernilai None setelah filter: {none_answers_after_filter}")

Filter:   0%|          | 0/16407 [00:00<?, ? examples/s]

Jumlah data setelah filter: 16407
Jumlah 'answer' yang bernilai None setelah filter: 0


In [ ]:
from datasets import load_dataset

medchat_qa = load_dataset("keivalya/MedQuad-MedicalQnADataset", split="train")
print(medchat_qa.column_names)
medchat_qa[1001]
# print("Jumlah entri:", len(medchat_qa))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

['qtype', 'Question', 'Answer']


{'qtype': 'research',
 'Question': 'what research (or clinical trials) is being done for Ataxia Telangiectasia ?',
 'Answer': 'NINDS-supported researchers discovered the gene responsible for A-T, known as ATM (ataxia-telangiectasia mutated) in 1995. This gene makes a protein that activates many (probably more than 700) other proteins that control cell cycle, DNA repair, and cell death. Without it, cells are unable to activate the cellular checkpoints that protect against the damage of ionizing radiation and other agents that can harm DNA. In addition to supporting basic research on A-T, NINDS also funds research aimed at A-T drug development, including development of animal models, gene and stem-cell based therapies, and high-throughput drug screens. The NINDS also leads a trans-NIH A-T Working Group whose members include NINDS, NHLBI, NIEHS, NCI, NEI, NIGMS, NHGRI, NIA, NIAID, NICHD, and ORD.'}

In [ ]:
from datasets import load_dataset
# MedQuAD
medquad = load_dataset("keivalya/MedQuad-MedicalQnADataset", split="train")
# MedChat-QA
medchat_qa = load_dataset("ngram/medchat-qa", split="train")
print(medquad.column_names)
print(medchat_qa.column_names)

README.md: 0.00B [00:00, ?B/s]

(…)gram-medchat-dataset-shuffled-v1.2.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/30238 [00:00<?, ? examples/s]

['qtype', 'Question', 'Answer']
['question', 'answer']


In [ ]:
# Hapus kolom qtype
medquad = medquad.remove_columns(["qtype"])
medquad = medquad.rename_columns({"Question": "question", "Answer": "answer"})

In [ ]:
from datasets import concatenate_datasets

merged_dataset = concatenate_datasets([medquad, medchat_qa])
# Shuffle with a fixed seed for reproducibility
merged_dataset = merged_dataset.shuffle(seed=42)
print("Total entries:", len(merged_dataset))

Total entries: 46645


In [ ]:
merged_dataset[22]

{'question': 'What does Enalapril Maleate inhibit in the body?',
 'answer': 'It inhibits the conversion of angiotensin I to angiotensin II, thereby preventing the vasoconstrictor and aldosterone-secreting effects of angiotensin II.'}

In [ ]:
# # Filter out examples where the instruction is too short
# initial_size = len(merged)
# filtered_dataset = merged.filter(lambda example: len(example['question']) > 3)
# # Filter out examples where the response is too short
# filtered_dataset = filtered_dataset.filter(lambda example: len(example['answer']) > 3)
# print(f"Original size: {initial_size}")
# print(f"Size after filtering: {len(filtered_dataset)}")

### Preprocessing Data

In [1]:
!pip install -U bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 27.9 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForLanguageModeling

# 1. Tentukan Nama Model & Muat Tokenizer
# Menggunakan model yang sama memastikan tokenizer cocok dengan arsitektur model.
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# --- PENTING: Atur Padding Token ---
# Model Mistral tidak memiliki padding token default.
# Cara umum adalah menggunakan end-of-sentence (eos) token sebagai padding token.
tokenizer.pad_token = tokenizer.eos_token

# 2. Muat Dataset
# Kita ambil sebagian kecil saja untuk demonstrasi agar cepat.
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset", split="train").select(range(10000))
print("Kolom asli:", dataset.column_names)
print("\nContoh data asli:")
print(dataset[0])

# Gunakan `train_test_split` untuk membagi dataset secara acak.
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

Kolom asli: ['qtype', 'Question', 'Answer']

Contoh data asli:
{'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?', 'Answer': 'LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.'}


In [4]:
def create_prompt(example):
    """
    Memformat example menjadi string prompt sesuai template Mistral Instruct.
    Template: <s>[INST] {instruksi} [/INST] {respons}</s>
    """
    # Pastikan tidak ada nilai None yang masuk ke dalam f-string
    question = example.get('Question', '') or ''
    qtype = example.get('qtype', '') or ''
    answer = example.get('Answer', '') or ''

    # Gabungkan question dan qtype sebagai instruksi
    instruction = f"{question.strip()} (context: {qtype.strip()})"

    # Buat prompt lengkap
    formatted_text = f"<s>[INST] {instruction} [/INST] {answer.strip()}</s>"

    return {"text": formatted_text}

# Aplikasikan fungsi formatting
train_formatted = train_dataset.map(create_prompt)
test_formatted = test_dataset.map(create_prompt)
print("\nContoh data setelah di-format:")
print(train_formatted[0]['text'])

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]


Contoh data setelah di-format:
<s>[INST] What are the genetic changes related to Erdheim-Chester disease ? (context: genetic changes) [/INST] More than half of people with Erdheim-Chester disease have a specific mutation in the BRAF gene. Mutations in other genes are also thought to be involved in this disorder.  The BRAF gene provides instructions for making a protein that helps transmit chemical signals from outside the cell to the cell's nucleus. This protein is part of a signaling pathway known as the RAS/MAPK pathway, which controls several important cell functions. Specifically, the RAS/MAPK pathway regulates the growth and division (proliferation) of cells, the process by which cells mature to carry out specific functions (differentiation), cell movement (migration), and the self-destruction of cells (apoptosis).  The BRAF gene mutation that causes Erdheim-Chester disease is somatic, which means that it occurs during a person's lifetime and is present only in certain cells. The

In [14]:
# 4. Fungsi Tokenisasi
# Mengubah teks yang sudah diformat menjadi token ID yang dimengerti model.
def tokenize_function(examples):
    # Tokenisasi teks dengan padding dan pemotongan (truncation) jika perlu.
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=128, # Sesuaikan max_length jika perlu
    )

# Aplikasikan tokenisasi secara batch untuk efisiensi
# Hapus kolom lama agar dataset bersih dan siap untuk training.
train_tokenized = train_formatted.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names + ['text'] # Hapus kolom 'question', 'answer', 'qtype', dan 'text'
)
test_tokenized = test_formatted.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names + ['text'] # Hapus kolom 'question', 'answer', 'qtype', dan 'text'
)
print("\nContoh data setelah di-tokenisasi:")
print(train_tokenized[0])

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]


Contoh data setelah di-tokenisasi:
{'input_ids': [1, 1, 28792, 16289, 28793, 1824, 460, 272, 19869, 4435, 5202, 298, 2852, 28715, 9733, 28733, 21124, 2406, 8030, 1550, 325, 2714, 28747, 19869, 4435, 28731, 733, 28748, 16289, 28793, 3764, 821, 2795, 302, 905, 395, 2852, 28715, 9733, 28733, 21124, 2406, 8030, 506, 264, 2948, 4548, 352, 297, 272, 365, 5244, 28765, 17198, 28723, 16969, 697, 297, 799, 24203, 460, 835, 1654, 298, 347, 5290, 297, 456, 17404, 28723, 28705, 415, 365, 5244, 28765, 17198, 5312, 11382, 354, 2492, 264, 14311, 369, 7263, 20689, 10845, 15972, 477, 3536, 272, 3601, 298, 272, 3601, 28742, 28713, 17336, 381, 28723, 851, 14311, 349, 744, 302, 264, 1492, 4726, 2439, 1128, 2651, 390, 272, 399, 2109, 28748, 10999, 28796, 2439, 1128, 28725, 690, 13186, 2856, 2278, 3601, 5572, 28723, 23518], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [15]:
# 5. Inisialisasi Data Collator
# Data collator akan menangani padding secara dinamis per batch.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Penting! Set mlm=False untuk Causal LM (seperti Mistral, GPT)
    )

In [7]:
!pip install trl
!pip install -U bitsandbytes

In [16]:
import torch
from transformers import (AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# 7. Konfigurasi Quantization (BitsAndBytes) untuk QLoRA
# Menggunakan 4-bit quantization untuk menghemat memori GPU secara signifikan.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # Tipe quantization 4-bit: Normal Float 4
    bnb_4bit_compute_dtype=torch.bfloat16, # Tipe data untuk komputasi
    bnb_4bit_use_double_quant=True,
)

# 8. Muat Model Dasar dengan Quantization
# Menggunakan AutoModelForCausalLM untuk memuat model Mistral dengan konfigurasi di atas.
# Kita matikan cache karena sering menyebabkan masalah pada fine-tuning.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    use_cache=False, # Penting untuk training
)
# Atur padding token di model juga
model.config.pad_token_id = tokenizer.pad_token_id

# 9. Siapkan Model untuk Training QLoRA
# Lakukan pre-processing agar model siap untuk training dengan metode QLoRA.
model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
# 10. Konfigurasi PEFT (Parameter-Efficient Fine-Tuning) LORA
# LoraConfig mendefinisikan parameter-parameter untuk LORA.
peft_config = LoraConfig(
    r=4, # Rank LORA. Makin besar, makin banyak parameter dan makin lambat trainingnya.
    lora_alpha=8, # Skala LORA.
    lora_dropout=0.0, # Dropout untuk regularisasi.
    bias="none", # Bias
    task_type="CAUSAL_LM", # Tipe task.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # Layer yang akan di-inject adapter LORA.
)

# Dapatkan model PEFT yang siap di-fine-tune
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


In [20]:
!accelerate config

--------------------------------------------------------------------------------In which compute environment are you running?
Please input a choice index (starting from 0), and press enter
 ➔  This machine
    AWS (Amazon SageMaker)
This machine
This machine
--------------------------------------------------------------------------------Which type of machine are you using?
Please input a choice index (starting from 0), and press enter
 ➔  No distributed training
    multi-CPU
    multi-XPU
    multi-HPU
    multi-GPU
    multi-NPU
    multi-MLU
    multi-SDAA
    multi-MUSA
    TPU
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/menu/cursor.py", line 63, in hide
    yield
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/menu/selection_menu.py", line 133, in run
    choice = int(builtins.input())
                 ^^^^^^^^^^^^^^^^
KeyboardInterrupt

During handling of the above exception, another exception occurred

In [18]:
0# Argumen Training
training_args = TrainingArguments(
    output_dir="./results", # Folder untuk menyimpan hasil
    num_train_epochs=3, # Jumlah epoch (sesuaikan jika perlu)
    per_device_train_batch_size=4, # Ukuran batch per GPU
    # gradient_accumulation_steps=2, # Mengakumulasi gradien untuk batch yang lebih besar
    learning_rate=2e-5, # Kecepatan belajar
    optim="paged_adamw_8bit", # Optimizer yang dioptimalkan untuk 8-bit
    lr_scheduler_type="cosine", # Scheduler untuk kecepatan belajar
    # warmup_ratio=0.03, # Rasio warmup untuk scheduler
    save_strategy="epoch", # Simpan model di setiap epoch
    eval_strategy="steps", # Evaluasi berdasarkan langkah
    fp16=True, # Menggunakan floating-point 16 bit saat train
    report_to='none'
)

# Inisialisasi SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    peft_config=peft_config,
    processing_class=tokenizer,
    args=training_args,
    data_collator=data_collator
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


KeyboardInterrupt: 